In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import os
os.environ["TF_GPU_ALLOCATOR"]='cuda_malloc_async'
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]='1.00'

In [8]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdullahkhan70/lumbar-spinal-mri-dataset")

print("Path to dataset files:", path)

100%|██████████| 52.2M/52.2M [00:00<00:00, 200MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1


In [1]:
# GPU 환경 확인
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report
import os

TensorFlow version: 2.19.0
GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [20]:
# 경로 설정 (kagglehub가 반환한 경로 그대로 사용)
base_dir = '/root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1/lumbar_spinal_dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# ImageDataGenerator 및 데이터 분할
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # train/val 분할
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 모델 구축
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=30
)

# 테스트셋 평가
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# 클래스 이름 출력
print("Class mapping:", train_generator.class_indices)

# 테스트 예측 및 리포트
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))

Found 3848 images belonging to 3 classes.
Found 960 images belonging to 3 classes.
Found 1158 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 59s 462ms/step - accuracy: 0.3925 - loss: 1.0930 - val_accuracy: 0.3958 - val_loss: 1.0879
Epoch 2/30
  1/120 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.5312 - loss: 1.0614

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.5312 - loss: 1.0614 - val_accuracy: 0.3958 - val_loss: 1.0905
Epoch 3/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 66s 431ms/step - accuracy: 0.3717 - loss: 1.0918 - val_accuracy: 0.3833 - val_loss: 1.0871
Epoch 4/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.4688 - loss: 1.1022 - val_accuracy: 0.3812 - val_loss: 1.0897
Epoch 5/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 71s 429ms/step - accuracy: 0.4208 - loss: 1.0787 - val_accuracy: 0.3760 - val_loss: 1.0869
Epoch 6/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.3750 - loss: 1.1036 - val_accuracy: 0.3771 - val_loss: 1.0883
Epoch 7/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 77s 468ms/step - accuracy: 0.4154 - loss: 1.0784 - val_accuracy: 0.3667 - val_loss: 1.0855
Epoch 8/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - accuracy: 0.3438 - loss: 1.0678 - val_accuracy: 0.3625 - val_loss: 1.0868
Epoch 9/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 72s 466ms/step - accuracy: 0.4289 - loss: 1.0750 - val_acc

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report
import numpy as np
import os
### ResNet50

# 경로 설정
base_dir = '/root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1/lumbar_spinal_dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# 데이터 제너레이터 준비
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),   # ResNet 입력 사이즈
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# ResNet50 - 사전학습된 모델 사용
resnet_base = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
resnet_base.trainable = False  # 처음엔 feature extractor(논문 권장 내용)

# 모델 작성
model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20   # ResNet은 일반적으로 빠른 수렴
)

# 테스트셋 평가
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")
print("Class mapping:", train_generator.class_indices)

# 성능 리포트
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))


Found 3848 images belonging to 3 classes.
Found 960 images belonging to 3 classes.
Found 1158 images belonging to 3 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 85s 604ms/step - accuracy: 0.3669 - loss: 1.3026 - val_accuracy: 0.4260 - val_loss: 1.0830
Epoch 2/20
  1/120 ━━━━━━━━━━━━━━━━━━━━ 13s 114ms/step - accuracy: 0.2500 - loss: 1.1819

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 93ms/step - accuracy: 0.2500 - loss: 1.1819 - val_accuracy: 0.3938 - val_loss: 1.0864
Epoch 3/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 56s 469ms/step - accuracy: 0.3698 - loss: 1.1121 - val_accuracy: 0.3885 - val_loss: 1.0843
Epoch 4/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - accuracy: 0.3438 - loss: 1.1310 - val_accuracy: 0.3771 - val_loss: 1.0845
Epoch 5/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 57s 472ms/step - accuracy: 0.3758 - loss: 1.0900 - val_accuracy: 0.3948 - val_loss: 1.0843
Epoch 6/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 21s 172ms/step - accuracy: 0.2188 - loss: 1.1431 - val_accuracy: 0.3948 - val_loss: 1.0856
Epoch 7/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 61s 474ms/step - accuracy: 0.3840 - loss: 1.0937 - val_accuracy: 0.4031 - val_loss: 1.0845
Epoch 8/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 96ms/step - accuracy: 0.4688 - loss: 1.0884 - val_accuracy: 0.4010 - val_loss: 1.0850
Epoch 9/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 71s 477ms/step - accuracy: 0.4041 - loss: 1.0877 - val_ac

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
### resnet50 (add preprocessing)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report
import numpy as np
import cv2
import os

# 1. 커스텀 전처리 함수 정의
def custom_preprocess(img):
    # img: RGB, float32, [0~255] 범위
    # 1) Grayscale 변환
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Resize
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    # 3) CLAHE (명암 보정)
    img = img.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    # 4) Gaussian Blur (노이즈 감소)
    img = cv2.GaussianBlur(img, (3,3), 0)
    # 5) 정규화: 0~1
    img = img.astype('float32') / 255.0
    # 6) 채널 확장 (ResNet50 입력: 3채널)
    img = np.expand_dims(img, axis=-1)    # (224,224,1)
    img = np.repeat(img, 3, axis=-1)      # (224,224,3)
    return img

# 2. 경로 세팅
base_dir = '/root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1/lumbar_spinal_dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# 3. ImageDataGenerator (커스텀 전처리 포함, rescale은 함수 내에서 처리)
train_gen = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=custom_preprocess,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(preprocessing_function=custom_preprocess)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 4. 사전학습 ResNet50 + 커스텀 출력 레이어
resnet_base = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
resnet_base.trainable = False       # feature extractor

model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5. 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20
)

# 6. 테스트셋 평가 및 리포트
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.4f}')
print('Class mapping:', train_generator.class_indices)

y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))


Found 3848 images belonging to 3 classes.
Found 960 images belonging to 3 classes.
Found 1158 images belonging to 3 classes.
Epoch 1/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 79s 568ms/step - accuracy: 0.3462 - loss: 1.2889 - val_accuracy: 0.3823 - val_loss: 1.0905
Epoch 2/20
  1/120 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - accuracy: 0.4375 - loss: 1.0723

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 21s 172ms/step - accuracy: 0.4375 - loss: 1.0723 - val_accuracy: 0.3948 - val_loss: 1.0877
Epoch 3/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 108s 496ms/step - accuracy: 0.3583 - loss: 1.1267 - val_accuracy: 0.3927 - val_loss: 1.0862
Epoch 4/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 98ms/step - accuracy: 0.3438 - loss: 1.0765 - val_accuracy: 0.3990 - val_loss: 1.0865
Epoch 5/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 70s 493ms/step - accuracy: 0.3558 - loss: 1.1025 - val_accuracy: 0.3958 - val_loss: 1.0869
Epoch 6/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.3438 - loss: 1.1186 - val_accuracy: 0.3958 - val_loss: 1.0866
Epoch 7/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 61s 510ms/step - accuracy: 0.4070 - loss: 1.0880 - val_accuracy: 0.3958 - val_loss: 1.0876
Epoch 8/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.4062 - loss: 1.0599 - val_accuracy: 0.3958 - val_loss: 1.0873
Epoch 9/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 68s 494ms/step - accuracy: 0.3929 - loss: 1.0866 - val

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
### Deep CNN

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import cv2
import os

# 1. 커스텀 전처리 함수 (앞서 오류 수정 반영)
def custom_preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.uint8)   # CLAHE 용
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.repeat(img, 3, axis=-1)
    return img

# 2. 경로 설정
base_dir = '/root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1/lumbar_spinal_dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# 3. ImageDataGenerator
train_gen = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=custom_preprocess,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(preprocessing_function=custom_preprocess)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 4. Deep CNN 모델 설계 (예시: Conv 블록 5개 이상)
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5. 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)

# 6. 테스트셋 평가 및 리포트
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.4f}')
print('Class mapping:', train_generator.class_indices)

y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))


Found 3848 images belonging to 3 classes.
Found 960 images belonging to 3 classes.
Found 1158 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 103s 666ms/step - accuracy: 0.3585 - loss: 2.1489 - val_accuracy: 0.3958 - val_loss: 1.1398
Epoch 2/25
  1/120 ━━━━━━━━━━━━━━━━━━━━ 23s 193ms/step - accuracy: 0.4375 - loss: 1.0826

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 99ms/step - accuracy: 0.4375 - loss: 1.0826 - val_accuracy: 0.3958 - val_loss: 1.1438
Epoch 3/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 94s 525ms/step - accuracy: 0.3955 - loss: 1.1709 - val_accuracy: 0.2833 - val_loss: 1.3064
Epoch 4/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 98ms/step - accuracy: 0.3438 - loss: 1.0499 - val_accuracy: 0.2833 - val_loss: 1.3088
Epoch 5/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 64s 536ms/step - accuracy: 0.4212 - loss: 1.0884 - val_accuracy: 0.3052 - val_loss: 1.9551
Epoch 6/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 98ms/step - accuracy: 0.3750 - loss: 1.0799 - val_accuracy: 0.3177 - val_loss: 1.9575
Epoch 7/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 69s 529ms/step - accuracy: 0.4075 - loss: 1.0722 - val_accuracy: 0.3948 - val_loss: 2.1560
Epoch 8/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 98ms/step - accuracy: 0.4062 - loss: 1.0718 - val_accuracy: 0.3625 - val_loss: 2.1371
Epoch 9/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 71s 534ms/step - accuracy: 0.4143 - loss: 1.0683 - val_acc

In [19]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

import os

# 경로 설정
base_dir = '/root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1/lumbar_spinal_dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# ImageDataGenerator (전처리 기본)
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Simple CNN 모델
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=30
)

# 테스트셋 평가
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# 클래스 이름 출력
print("Class mapping:", train_generator.class_indices)

# 테스트 예측
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

# 분류 보고서
target_names = list(train_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

# 혼동행렬
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# 민감도(Recall) & 특이도(Specificity) 직접 계산
sens_list = []
spec_list = []
for i in range(len(target_names)):
    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    FP = np.sum(cm[:, i]) - TP
    TN = np.sum(cm) - (TP + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    sens_list.append(sensitivity)
    spec_list.append(specificity)
    print(f"Class [{target_names[i]}] - Sensitivity(Recall): {sensitivity:.4f}, Specificity: {specificity:.4f}")

# 전체 평균 민감도/특이도
print(f"Mean Sensitivity(Recall): {np.mean(sens_list):.4f}")
print(f"Mean Specificity: {np.mean(spec_list):.4f}")


Found 3848 images belonging to 3 classes.
Found 960 images belonging to 3 classes.
Found 1158 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 74s 577ms/step - accuracy: 0.3742 - loss: 1.1050 - val_accuracy: 0.3958 - val_loss: 1.0906
Epoch 2/30
  1/120 ━━━━━━━━━━━━━━━━━━━━ 8s 69ms/step - accuracy: 0.4375 - loss: 1.0933

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step - accuracy: 0.4375 - loss: 1.0933 - val_accuracy: 0.3958 - val_loss: 1.0935
Epoch 3/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 112s 444ms/step - accuracy: 0.3798 - loss: 1.0938 - val_accuracy: 0.3948 - val_loss: 1.0891
Epoch 4/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - accuracy: 0.4062 - loss: 1.0732 - val_accuracy: 0.3958 - val_loss: 1.0863
Epoch 5/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 53s 442ms/step - accuracy: 0.4078 - loss: 1.0846 - val_accuracy: 0.3448 - val_loss: 1.0913
Epoch 6/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - accuracy: 0.4688 - loss: 1.0741 - val_accuracy: 0.3458 - val_loss: 1.0894
Epoch 7/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 71s 435ms/step - accuracy: 0.4043 - loss: 1.0791 - val_accuracy: 0.3719 - val_loss: 1.0859
Epoch 8/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - accuracy: 0.3750 - loss: 1.0762 - val_accuracy: 0.3635 - val_loss: 1.0875
Epoch 9/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 53s 438ms/step - accuracy: 0.4280 - loss: 1.0699 - val_ac

In [21]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import os

# 1. 커스텀 전처리 함수 (CLAHE 오류 수정 포함)
def custom_preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.uint8)   # CLAHE 용
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.repeat(img, 3, axis=-1)
    return img

# 2. 경로 설정
base_dir = '/root/.cache/kagglehub/datasets/abdullahkhan70/lumbar-spinal-mri-dataset/versions/1/lumbar_spinal_dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# 3. ImageDataGenerator
train_gen = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=custom_preprocess,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(preprocessing_function=custom_preprocess)

train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 4. Deep CNN 모델 설계
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5. 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)

# 6. 테스트셋 평가 및 성능 출력
test_loss, test_acc = model.evaluate(test_generator)
print(f'\nTest accuracy (전체): {test_acc:.4f}')
print('Class mapping:', train_generator.class_indices)

y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

# 분류 보고서 (precision, recall 등)
target_names = list(train_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=target_names))

# 혼동행렬 및 민감도/특이도 직접 계산
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", cm)

print("\nClass별 민감도(Recall) 및 특이도(Specificity):")
sens_list = []
spec_list = []
for i in range(len(target_names)):
    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    FP = np.sum(cm[:, i]) - TP
    TN = np.sum(cm) - (TP + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0      # 민감도 = recall
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0      # 특이도 정의
    sens_list.append(sensitivity)
    spec_list.append(specificity)
    print(f"  [{target_names[i]}]: Sensitivity(Recall)={sensitivity:.4f}, Specificity={specificity:.4f}")

print(f"\nMean Sensitivity(Recall): {np.mean(sens_list):.4f}")
print(f"Mean Specificity: {np.mean(spec_list):.4f}")
print(f"\nTest accuracy (전체): {test_acc:.4f}")


Found 3848 images belonging to 3 classes.
Found 960 images belonging to 3 classes.
Found 1158 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 91s 663ms/step - accuracy: 0.3542 - loss: 1.9981 - val_accuracy: 0.3958 - val_loss: 1.1071
Epoch 2/25
  1/120 ━━━━━━━━━━━━━━━━━━━━ 26s 225ms/step - accuracy: 0.3438 - loss: 1.3477

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 109ms/step - accuracy: 0.3438 - loss: 1.3477 - val_accuracy: 0.3958 - val_loss: 1.1102
Epoch 3/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 69s 575ms/step - accuracy: 0.3718 - loss: 1.1754 - val_accuracy: 0.3958 - val_loss: 1.2667
Epoch 4/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 108ms/step - accuracy: 0.5000 - loss: 1.0207 - val_accuracy: 0.3958 - val_loss: 1.2630
Epoch 5/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 129s 574ms/step - accuracy: 0.3797 - loss: 1.1062 - val_accuracy: 0.3958 - val_loss: 1.5197
Epoch 6/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 108ms/step - accuracy: 0.4688 - loss: 1.0537 - val_accuracy: 0.3958 - val_loss: 1.5249
Epoch 7/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 70s 583ms/step - accuracy: 0.3955 - loss: 1.0740 - val_accuracy: 0.3927 - val_loss: 1.6509
Epoch 8/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 109ms/step - accuracy: 0.3750 - loss: 1.0781 - val_accuracy: 0.3969 - val_loss: 1.6508
Epoch 9/25
120/120 ━━━━━━━━━━━━━━━━━━━━ 69s 576ms/step - accuracy: 0.4076 - loss: 1.0612 - va